# Costs and Time Overviews
## Costs
* Started with $5.58  
* Ended up with $5.32 after 100 books 
* So $0,26 per 100 books
* So for 17,000 books = $44,2

## Time
* 100 books take around 5 minutes, 
* so 17,000 books will take around 14 hours

In [ ]:
# # Find 20 novels that I've read to check manually if FMC extraction works

# # Original data with an extra column
# books = [
#     {"Title": "Pride and Prejudice", "Author": "Jane Austen", "Female_Major_Character": 1},  # Elizabeth Bennet
#     {"Title": "To Kill a Mockingbird", "Author": "Harper Lee", "Female_Major_Character": 1},  # Scout Finch
#     {"Title": "The Book Thief", "Author": "Markus Zusak", "Female_Major_Character": 1},  # Liesel Meminger
#     {"Title": "Twilight", "Author": "Stephenie Meyer", "Female_Major_Character": 1},  # Bella Swan
#     {"Title": "The Fault in Our Stars", "Author": "John Green", "Female_Major_Character": 1},  # Hazel Grace Lancaster
#     {"Title": "The Perks of Being a Wallflower", "Author": "Stephen Chbosky", "Female_Major_Character": 1},  # Sam
#     {"Title": "Brave New World", "Author": "Aldous Huxley", "Female_Major_Character": 1}, #Lenina Crowne
#     {"Title": "A Game of Thrones (A Song of Ice and Fire, #1)", "Author": "George R.R. Martin", "Female_Major_Character": 1},  # Daenerys Targaryen, Arya Stark, Cersei Lannister, ...
#     {"Title": "The Lightning Thief", "Author": "Rick Riordan", "Female_Major_Character": 1},  # Annabeth Chase
#     {"Title": "Lolita", "Author": "Vladimir Nabokov", "Female_Major_Character": 1},  # Dolores "Lolita" Haze

#     {"Title": "The Hobbit and The Lord of the Rings", "Author": "J.R.R. Tolkien", "Female_Major_Character": 0},
#     {"Title": "The Little Prince", "Author": "Antoine de Saint-Exupéry", "Female_Major_Character": 0},
#     {"Title": "Crime and Punishment", "Author": "Fyodor Dostoevsky", "Female_Major_Character": 0},  
#     {"Title": "Lord of the Flies", "Author": "William Golding", "Female_Major_Character": 0}, 
#     {"Title": "The Adventures of Huckleberry Finn", "Author": "Mark Twain", "Female_Major_Character": 0}, 
#     {"Title": "The Old Man and the Sea", "Author": "Ernest Hemingway", "Female_Major_Character": 0}, 
#     {"Title": "Alex Rider", "Author": "Anthony Horowitz", "Female_Major_Character": 0},  
#     {"Title": "Der Trafikant", "Author": "Robert Seethaler", "Female_Major_Character": 0}, 
# ]

# # Create the DataFrame
# df_books = pd.DataFrame(books)

# # Display the updated DataFrame
# print(df_books)

In [ ]:
# Iterating through books, 1 book = 1 request
for index, row in df100.iterrows():
    # Extract title and author from each row 
    title = row["title"]
    author = row["author"]

    # Format user prompt with current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)

    # Payload
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 200,
        "temperature": 0,  # Controls randomness in the response
        # "top_p": 0.9,
        # "search_domain_filter": ["<any>"],
        # "return_images": False,
        # "return_related_questions": False,
        # "search_recency_filter": "<string>",
        # "top_k": 0,
        # "stream": False,
        # "presence_penalty": 0,
        # "frequency_penalty": 1,
        "response_format": {
          "type": "json_schema",
          "json_schema": {"schema": AnswerFormat.model_json_schema()},
        },
        # "web_search_options": {"search_context_size": "high"}
    }

    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        # Extract sources
        citation = response_json.get("citations", [])
        # Store answer and sources
        responses.append(answer)
        citations.append(citation)
    else:
        # Handle errors by appending an error message
        responses.append(f"Error: {response.status_code}")
    
    # # Sleep to avoid rate limiting
    # time.sleep(1)

# End timing the request process
end_time = time.time()

# Calculate and print the time taken for the 100 requests
elapsed_time = end_time - start_time
print(f"Time taken for 100 books: {elapsed_time} seconds")